In [ ]:
import requests
from fastapi import FastAPI
from datetime import datetime, timedelta
import pickle
import numpy as np
import pandas as pd
from apscheduler.schedulers.background import BackgroundScheduler
from contextlib import asynccontextmanager
import json
from pathlib import Path

@asynccontextmanager
async def lifespan(app: FastAPI):
    print("Scheduler started.")
    yield

app = FastAPI(lifespan=lifespan)

NYS_NYC_COORDS = (40.7128, -74.0060)

weather_pass = 'mmmv viyv apvo hgnz'
weather_name = 'weather-0'

# Load model once
with open("xgb_temp_model.pkl", "rb") as f:
    model = pickle.load(f)

# Load precomputed average absolute error CSV once
error_df = pd.read_csv("avg_abs_error_by_time.csv")

LOG_FILE = Path("forecast_log.jsonl")

def update_forecast_log(current_time, actual_temp, predicted_time, predicted_temp):
    # Load current data
    try:
        with open(LOG_FILE, "r") as f:
            entries = [json.loads(line) for line in f if line.strip()]
    except FileNotFoundError:
        entries = []

    # Turn into a dict
    entry_map = {e["time"]: e for e in entries}

    # Ensure both entries exist
    current_key = current_time.isoformat()
    future_key = predicted_time.isoformat()

    entry_map.setdefault(current_key, {"time": current_key, "actual": None, "pred": None})
    entry_map.setdefault(future_key, {"time": future_key, "actual": None, "pred": None})

    # Safe float conversion
    try:
        entry_map[current_key]["actual"] = float(actual_temp)
    except (TypeError, ValueError):
        pass

    try:
        entry_map[future_key]["pred"] = float(predicted_temp)
    except (TypeError, ValueError):
        pass

    # Write back all updated entries
    with open(LOG_FILE, "w") as f:
        for entry_time, e in sorted(entry_map.items()):
            f.write(json.dumps(e) + "\n")

from fastapi.responses import JSONResponse

@app.get("/forecast-log")
def get_forecast_log():
    if not LOG_FILE.exists():
        return JSONResponse(content={"message": "Log file not found."}, status_code=404)
    
    entries = []
    with open(LOG_FILE, "r") as f:
        for line in f:
            entries.append(json.loads(line))
    
    return entries

from datetime import datetime, timedelta

@app.get("/nyc-weather")
def get_nyc_weather():

    lat, lon = NYS_NYC_COORDS
    points_url = f"https://api.weather.gov/points/{lat},{lon}"
    points_data = requests.get(points_url).json()

    forecast_url = points_data["properties"]["forecast"]
    forecast_data = requests.get(forecast_url).json()

    first_period = forecast_data["properties"]["periods"][0]
    
    # Parse start_time string to datetime object
    start_time = datetime.fromisoformat(first_period["startTime"].replace("Z", "+00:00"))
    current_temp = first_period["temperature"]

    hourly_predictions = []
    for i in range(24):
        future_time = start_time + timedelta(hours=i)
        month = future_time.month
        day = future_time.day
        hour = future_time.hour

        X = np.array([[month, day, hour, current_temp]])
        predicted_temp = model.predict(X)[0]

        matched = error_df[
            (error_df['month'] == month) &
            (error_df['day'] == day) &
            (error_df['hour'] == hour)
        ]

        expected_abs_error = matched['abs_error'].values[0] if not matched.empty else 'N/A'

        hourly_predictions.append({
            "time": future_time.isoformat(),
            "predicted_temperature": predicted_temp,
            "expected_abs_error": expected_abs_error
        })

    # Log times also in ISO format
    update_forecast_log(start_time, current_temp, start_time + timedelta(hours=1), hourly_predictions[0]['predicted_temperature'])

    return {
        "forecast_name": first_period["name"],
        "current_temperature": current_temp,
        "temperature_unit": first_period["temperatureUnit"],
        "forecast_short": first_period["shortForecast"],
        "forecast_detailed": first_period["detailedForecast"],
        "start_time": start_time.isoformat(),
        "predictions_next_24_hours": hourly_predictions
    }

from email.message import EmailMessage
import smtplib

EMAIL_FROM = "cosenzazachary@gmail.com"
EMAIL_TO = "cosenzazachary@gmail.com"
EMAIL_PASSWORD = weather_pass

import matplotlib.pyplot as plt
from io import BytesIO
from email.mime.image import MIMEImage
from email.utils import make_msgid

@app.get("/email-forecast")
def email_forecast():

    # Step 1: Get 24-hour forecast data
    total_forecast = get_nyc_weather()  # returns list of dicts with datetime, predicted_temperature, expected_abs_error

    hourly_forecast = total_forecast['predictions_next_24_hours']

    df = pd.DataFrame(hourly_forecast)
    times = df['time']
    temps = df['predicted_temperature']
    errors = df['expected_abs_error']

    # Step 2: Generate matplotlib plot
    plt.figure(figsize=(10, 4))
    plt.plot(times, temps, label="Predicted Temp", color='tab:blue', marker='o')

    # Add thick red error bars centered on predictions
    plt.errorbar(times, temps, yerr=errors, fmt='none', ecolor='red', elinewidth=3, capsize=4, label="± Error")

    plt.title("24-Hour Temperature Forecast")
    plt.xlabel("Time")
    plt.ylabel("Temperature (°F)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.legend()

    img_buf = BytesIO()
    plt.savefig(img_buf, format="png")
    img_buf.seek(0)
    plt.close()

    # Generate Content-ID
    image_cid = make_msgid(domain='xyz.com')[1:-1]  # Strip angle brackets

    # Step 3: Create and send email
    msg = EmailMessage()
    msg.set_content("See attached image for the 24-hour forecast.")
    msg['Subject'] = "NYC Forecast and Model Prediction"
    msg['From'] = EMAIL_FROM
    msg['To'] = EMAIL_TO

    # Plain text fallback
    msg.set_content("Your device does not support HTML emails. Forecast image attached.")

    msg.add_alternative(f"""
        <html>
            <body>
                <h2 style="color:navy;">NYC 24-Hour Forecast</h2>
                
                <p><b>Summary:</b> Attached is the latest prediction with expected error ranges.</p>

                <p>
                    <b>Highlights:</b><br>
                    &bull; Temperatures updated hourly<br>
                    &bull; Includes ± expected error range<br>
                    &bull; All times in Eastern Time
                </p>

                <p style="margin-top: 20px;">
                    <img src="cid:{image_cid}" alt="Forecast Plot" style="max-width: 600px; border: 1px solid #ccc;">
                </p>

                <p style="font-size: 0.9em; color: gray;">
                    This forecast was generated on {datetime.now().strftime('%B %d, %Y at %I:%M %p')}.
                </p>
            </body>
        </html>
    """, subtype='html')

    # Attach image inline
    image = MIMEImage(img_buf.read(), _subtype='png')
    image.add_header('Content-ID', f'<{image_cid}>')
    image.add_header('Content-Disposition', 'inline', filename="forecast.png")
    msg.get_payload()[1].add_related(image)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(EMAIL_FROM, EMAIL_PASSWORD)
        smtp.send_message(msg)

    return {"message": "Email with image sent successfully!"}

# Scheduler setup
scheduler = BackgroundScheduler()
scheduler.add_job(email_forecast, "interval", minutes = 30)  # Customize frequency
scheduler.start()